# **Ingestão de Dados para o Stage Bronze no Databricks**

### **Descrição do Processo**  
Este processo tem como objetivo realizar a ingestão de dados no **stage Bronze** da arquitetura Medallion. Ele lê dados em formato Parquet de um **Data Lake**, adiciona uma coluna de **timestamp** e salva as informações em um banco de dados Delta.

---

### **Origem do Processo**  
Os dados são lidos a partir de arquivos **Parquet** localizados no **Data Lake**, processados com a adição de um **timestamp** e armazenados em uma tabela **Delta** no banco de dados especificado.

---

### **Etapas do Processamento**


1. **`CMD 2 - Execução de Script de Parâmetros`**  
   O comando `%run ./Parametros` executa o script de parâmetros, que carrega a lista `parameters` contendo as configurações de leitura e ingestão de dados (como `read_path`, `table_name` e `database_name`).

2. **`CMD 3 - Importação de Funções do PySpark`**  
   As funções necessárias são importadas do módulo `pyspark.sql.functions`:  
   - `current_timestamp()`: Para adicionar a coluna de **timestamp** com a data e hora atuais.
   - `col()`: Utilizada para referenciar colunas dentro de expressões (não utilizada diretamente neste código, mas é uma função útil para manipulação de colunas).


3. **`CMD 4 - Comando SQL para Criação de Banco de Dados`**  
   - Este comando SQL, comentado, seria responsável pela criação do banco de dados **`adventure_works_bronze`** se ele não existir. A localização do banco de dados seria no **Data Lake**.
   - O código está comentado para não ser executado automaticamente.

4. **`CMD 5 - Definição da Função `ingest_data_bronze`**  
   A função `ingest_data_bronze` é responsável pela ingestão de dados do **stage Bronze**:
   - **Parâmetros**:
     - `read_path`: Caminho do arquivo Parquet no Data Lake.
     - `table_name`: Nome da tabela que será criada no banco de dados Delta.
     - `database_name`: Nome do banco de dados onde a tabela será salva.
   - **Processamento**:
     - Leitura do arquivo Parquet.
     - Adição da coluna `timestamp_bronze` com o timestamp atual.
     - Salvamento no formato **Delta**, sobrescrevendo a tabela existente, caso necessário.
     - Em caso de erro, uma mensagem indicando a falha no processamento é impressa.

5. **`CMD 6 - Execução da Função para Todos os Parâmetros**  
   O código itera sobre a lista `parameters`, chamando a função `ingest_data_bronze` para cada conjunto de parâmetros definidos.  
   Após o processamento de todos os dados, a mensagem "Processamento finalizado" é exibida.

---

### **Destino do Processo**  
Após a execução, as tabelas **Delta** são salvas no banco de dados especificado, na camada **Bronze**, tornando os dados acessíveis para processamento posterior.

---

### **Observações Importantes**
- A função **`ingest_data_bronze`** é robusta e pode lidar com vários conjuntos de dados de forma flexível, de acordo com os parâmetros fornecidos.
- A tabela Delta é sobrescrita caso já exista, utilizando o modo `overwrite`.
- A adição da coluna `timestamp_bronze` permite rastrear o momento exato da ingestão dos dados.
- O processo pode ser repetido sempre que necessário, desde que o caminho de leitura e os parâmetros estejam corretamente configurados.



In [0]:
%run ./Config/Parameters

In [0]:
%run ./Config/LogProcessamento

In [0]:
from pyspark.sql.functions import current_timestamp, col
from concurrent.futures import ThreadPoolExecutor

dbutils.widgets.text('Tabela', '')


In [0]:
%sql

USE CATALOG hive_metastore;

CREATE DATABASE IF NOT EXISTS adventure_works_bronze
LOCATION '/mnt/adlsstoragemasterprd/landing-zone/MedallionArchitecture/bronze/adventure_works_bronze';

CREATE DATABASE IF NOT EXISTS adventure_works_silver
LOCATION '/mnt/adlsstoragemasterprd/landing-zone/MedallionArchitecture/silver/adventure_works_silver';

CREATE DATABASE IF NOT EXISTS adventure_works_gold
LOCATION '/mnt/adlsstoragemasterprd/landing-zone/MedallionArchitecture/gold/adventure_works_gold';


In [0]:
def ingest_data_bronze(read_path, table_name, database_name):
    """
    Ingestão de dados para o stage bronze no formato bruto
        - Ler parquet do datalake
        - Adicionar coluna timestamp_bronze
        - Salvar em delta no banco de dados
    """
    try:
        df = spark.read.parquet(read_path)
        df.write.mode('overwrite').option('overwriteSchema', True).format('delta').saveAsTable(f"{database_name}.{table_name}")
        print(f"✅ tabela {table_name} Processada com sucesso")
    except:
        print(f"❌ tabela {table_name} Falhou")

In [0]:
tabela = dbutils.widgets.get('Tabela')
log_data = {
    "log_tabela": tabela,
    "log_camada": "Bronze",
    "log_origem": "adventure_works_bronze",
    "log_destino": "adventure_works_bronze",
}


In [0]:
tabela = dbutils.widgets.get('Tabela')

if tabela != None and tabela != "":
    for i in parameters:
        if i["table_name"] == tabela:
            try:
                addlog(**log_data, log_status='Sucesso',atualizacao=0)
                ingest_data_bronze(i['read_path'], i['table_name'], i['database_name'])
                addlog(**log_data, log_status='Sucesso',atualizacao=1)
            except:
                addlog(**log_data, log_status='Falha',atualizacao=1)
    dbutils.notebook.exit(f"✅ tabela {tabela} Processada com sucess")

In [0]:
with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(lambda p: ingest_data_bronze(p['read_path'], p['table_name'], p['database_name']), parameters)

print("Processamento finalizado")